In [4]:
import csv
import re

In [1]:
with open('jun_2020_sec_scan.csv', 'rb') as csvin, open('jun_2020_sec_scan_upd.csv','wb') as csvout:
    reader = csv.DictReader(csvin)
    print reader.fieldnames
    #print len(list(reader))
    
    fieldnames = reader.fieldnames + ['Product'] + ['Env'] + ['NetBIOS1'] + ['FI'] + ['Tier'] + ['Title1'] + ['Title2'] + ['Title3']
    writer = csv.DictWriter(csvout, fieldnames)

    writer.writeheader()    
      
    for row in reader:
        short_NetBIOS ='Unknown'
        Tier='Unknown'
        short_Title='Unknown'
        short_DNS='Unknown'
        Title1='Unknown'
        Title2='Unknown'
        Title3='Unknown'
        Env='Unknown'
        Product='Unknown'
        FI='Unknown'
        NetBIOS1=row['IP']
        
        if row:
            #print row
            #print row['NetBIOS']

            # Splitting Title to get Sub titles
            if row['Title']:
                short_Title = re.split('\(',row['Title'])

            # Generate Short DNS names
            if not re.match(r"^(NULL|NA|\s+)",row['DNS']):
                short_DNS = row['DNS'].split('.')[0]
                
            
            # Dissect NetBIOS for Product Name, Tier, environment, FI
            if not re.match(r"^(NULL$|NA$|\s*$)",row['NetBIOS']):
                NetBIOS_split = row['NetBIOS'].split('-')
                Product = NetBIOS_split[0]
                NetBIOS1 = row['NetBIOS']
                 
                if re.match(r"ngi|pbsfid|cs|inf|elk|cags|bfs|dgba", Product):
                    Env = NetBIOS_split[1]
                    Tier = NetBIOS_split[-1]
                elif Product == "cfrm":
                    m = row['NetBIOS'].split('-')
                    if re.match('^bfs|share', m[1]):
                        FI = m[1]
                        Env = m[2]
                        Tier = m[3]
                    elif re.match('^dgbcs', m[1]):
                        if re.match('^dc4', m[2]):
                            FI = m[1]
                            Env = m[3]
                            Tier = m[4] + "-" + m[5]
                        else:
                            FI = m[1]
                            Env = m[2]
                            try:
                                Tier = m[3] + "-" + m[4]
                            except:
                                Tier = m[3]
                    elif re.match('^pr$', m[1]):
                        Env = m[1]
                        FI = m[2]
                        Tier = m[3]
                    else:
                        n = re.match('^(pr)(\d+)', m[1])
                        Env = n.group(0)
                        FI = n.group(1)
                        Tier = m[3]
                elif Product == "dgb":
                    m = re.match('^dgb-(\w+)-(bbva|fi\d+|shared)*-*(\w+)', row['NetBIOS'])
                    Env = m.group(1)
                    if m > 3:
                        FI = m.group(2)
                        Tier = m.group(3)
                    else:
                        Tier = m.group(2)
                elif Product == "ws":
                    m = re.match('^ws-(\w+)-(dr|bmo|citi|jll|fi\d+)-*(\w+)', row['NetBIOS'])
                    Env = m.group(1)
                    FI = m.group(2)
                    Tier = m.group(3)
                elif Product == "wpf":
                    m = re.match('^wpf-(\w+)-(\w+)*-*(\w+)', row['NetBIOS'])
                    Env = m.group(1)
                    if m.group(3):
                        FI = m.group(2)
                        Tier = m.group(3)
                    else:
                        Tier = m.group(2)
                else:
                    Env = "Unknown"  
                    Tier = "Unknown"
            
            elif short_DNS:
                NetBIOS1 = short_DNS
                print "======>",NetBIOS1
            else:
                NetBIOS1 = row['IP']
                print "======>",NetBIOS1
                
            
            #print Product, Env, FI, Tier
            # If SubTitle 1 exist, then subtitle 1
            if short_Title[0]:
                Title1 = re.sub('\s+','',short_Title[0])
                #print Title1,
            
            # If subtitle greater than 1, then set further subtitle
            if len(short_Title) > 1: 
                Title2 = re.sub('\s|\)','',short_Title[1])
                #Title3 = re.split(':','',Title2)
                #print Title2
                
                # Classify subtitle 3 for some non standard subtitle
                if re.match('ADV', Title2):
                    m = re.match(r"(\D+)(\d+)", Title2)
                    Title3 = m.group(1) 
                elif re.match('MS|VMSA', Title2):
                    Title3 = Title2.split('-')[0]
                elif re.match('RHSA', Title2):
                    Title3 = Title2.split(':')[0]
                elif re.match('Sweet|Sad', Title2):
                    Title3 = re.sub('\s+','',Title2)
                elif re.match('ibm|KB', Title2):
                    m = re.match(r"(\D+)\d+",Title2)
                    Title3 = m.group(1)
                else:
                    m = re.match(r"(\w{5})*",Title2)
                    Title3 = m.group(1)
            else:
                m = re.match(r"(^\S{5})*",Title1)
                Title3 = m.group(1)
                   
            #print row['IP'],row['DNS'],row['NetBIOS'],Product,Env,NetBIOS1,FI,Tier,Title1,Title2,Title3
            # Write csv file out
            writer.writerow(dict(row, Product=Product, Env=Env, NetBIOS1=NetBIOS1, FI=FI, Tier=Tier, Title1=Title1, Title2=Title2, Title3=Title3))

        else:
            print 

NameError: name 'csv' is not defined

In [6]:
csvin.close()
csvout.close()